In [1]:
from peft import LoraConfig, get_peft_model
config = LoraConfig

In [4]:
from transformers import AutoModelForCausalLM
import torch
# 查看模型所有可训练参数，在后面加载lora的traget_module时可以选择
from transformers import AutoTokenizer
file = '../download_llm/Qwen-1_8B-Chat'
model = AutoModelForCausalLM.from_pretrained(
    file,
    trust_remote_code=True
)

for name, parameter in model.named_parameters():
    print(name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

transformer.wte.weight
transformer.h.0.ln_1.weight
transformer.h.0.attn.c_attn.weight
transformer.h.0.attn.c_attn.bias
transformer.h.0.attn.c_proj.weight
transformer.h.0.ln_2.weight
transformer.h.0.mlp.w1.weight
transformer.h.0.mlp.w2.weight
transformer.h.0.mlp.c_proj.weight
transformer.h.1.ln_1.weight
transformer.h.1.attn.c_attn.weight
transformer.h.1.attn.c_attn.bias
transformer.h.1.attn.c_proj.weight
transformer.h.1.ln_2.weight
transformer.h.1.mlp.w1.weight
transformer.h.1.mlp.w2.weight
transformer.h.1.mlp.c_proj.weight
transformer.h.2.ln_1.weight
transformer.h.2.attn.c_attn.weight
transformer.h.2.attn.c_attn.bias
transformer.h.2.attn.c_proj.weight
transformer.h.2.ln_2.weight
transformer.h.2.mlp.w1.weight
transformer.h.2.mlp.w2.weight
transformer.h.2.mlp.c_proj.weight
transformer.h.3.ln_1.weight
transformer.h.3.attn.c_attn.weight
transformer.h.3.attn.c_attn.bias
transformer.h.3.attn.c_proj.weight
transformer.h.3.ln_2.weight
transformer.h.3.mlp.w1.weight
transformer.h.3.mlp.w2.weight

In [6]:
model

QWenLMHeadModel(
  (transformer): QWenModel(
    (wte): Embedding(151936, 2048)
    (drop): Dropout(p=0.0, inplace=False)
    (rotary_emb): RotaryEmbedding()
    (h): ModuleList(
      (0-23): 24 x QWenBlock(
        (ln_1): RMSNorm()
        (attn): QWenAttention(
          (c_attn): Linear(in_features=2048, out_features=6144, bias=True)
          (c_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (attn_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): RMSNorm()
        (mlp): QWenMLP(
          (w1): Linear(in_features=2048, out_features=5504, bias=False)
          (w2): Linear(in_features=2048, out_features=5504, bias=False)
          (c_proj): Linear(in_features=5504, out_features=2048, bias=False)
        )
      )
    )
    (ln_f): RMSNorm()
  )
  (lm_head): Linear(in_features=2048, out_features=151936, bias=False)
)

In [12]:
a = ['1']
a[-1]

'1'

In [11]:
import torch.nn as nn
cls =  nn.Linear
for name, module in model.named_modules():
    if isinstance(module, cls):
            names = name.split('.')
            print(names)

['transformer', 'h', '0', 'attn', 'c_attn']
['transformer', 'h', '0', 'attn', 'c_proj']
['transformer', 'h', '0', 'mlp', 'w1']
['transformer', 'h', '0', 'mlp', 'w2']
['transformer', 'h', '0', 'mlp', 'c_proj']
['transformer', 'h', '1', 'attn', 'c_attn']
['transformer', 'h', '1', 'attn', 'c_proj']
['transformer', 'h', '1', 'mlp', 'w1']
['transformer', 'h', '1', 'mlp', 'w2']
['transformer', 'h', '1', 'mlp', 'c_proj']
['transformer', 'h', '2', 'attn', 'c_attn']
['transformer', 'h', '2', 'attn', 'c_proj']
['transformer', 'h', '2', 'mlp', 'w1']
['transformer', 'h', '2', 'mlp', 'w2']
['transformer', 'h', '2', 'mlp', 'c_proj']
['transformer', 'h', '3', 'attn', 'c_attn']
['transformer', 'h', '3', 'attn', 'c_proj']
['transformer', 'h', '3', 'mlp', 'w1']
['transformer', 'h', '3', 'mlp', 'w2']
['transformer', 'h', '3', 'mlp', 'c_proj']
['transformer', 'h', '4', 'attn', 'c_attn']
['transformer', 'h', '4', 'attn', 'c_proj']
['transformer', 'h', '4', 'mlp', 'w1']
['transformer', 'h', '4', 'mlp', 'w2'

## Lora Test

In [1]:
import torch
print(torch.cuda.is_available())

True


In [7]:
from peft import LoraConfig, TaskType, get_peft_model
config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    target_modules=["c_attn", "c_proj", "w1", "w2"],  # 这个不同的模型需要设置不同的参数，需要看模型中的attention层
    inference_mode=False,  # 训练模式
    r=8,  # Lora 秩
    lora_alpha=32,  # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1  # Dropout 比例
)
# 加载lora参数
model = get_peft_model(model, config)
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): QWenLMHeadModel(
      (transformer): QWenModel(
        (wte): Embedding(151936, 2048)
        (drop): Dropout(p=0.0, inplace=False)
        (rotary_emb): RotaryEmbedding()
        (h): ModuleList(
          (0-23): 24 x QWenBlock(
            (ln_1): RMSNorm()
            (attn): QWenAttention(
              (c_attn): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=6144, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=6144, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
 